In [2]:
import re
import json
import pytz
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

---

### Dados da B3 (Dados das Empresas Listadas)

**Colunas de interesse**

- Nome Pregão
- Tickers

In [3]:
with open('../crawlers/b3/data/results-b3.json', encoding='utf8') as json_file:
    data = json.load(json_file)

df_b3 = pd.DataFrame(data)

In [4]:
df_b3.sample(5)

,razao_social,nome_pregao,segmento,tickers,setor_economico,subsetor,segmento_setorial,url_dados
331,CVC BRASIL OPERADORA E AGÊNCIA DE VIAGENS S.A.,CVC BRASIL,NM,[CVCB3],Consumo Cíclico,Viagens e Lazer,Viagens e Turismo,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
140,PET CENTER COMERCIO E PARTICIPACOES S.A.,PETZ,NM,[PETZ3],Consumo Cíclico,Comércio,Produtos Diversos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
382,CIA GAS DE SAO PAULO - COMGAS,COMGAS,NDA,"[CGAS3, CGAS5]",Utilidade Pública,Gás,Gás,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
154,OPPORTUNITY ENERGIA E PARTICIPACOES S.A.,OPPORT ENERG,MB,[OPHE3B],Outros,Outros,Outros,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
307,ELETROPAULO METROP. ELET. SAO PAULO S.A.,ELETROPAULO,NDA,[],Utilidade Pública,Energia Elétrica,Energia Elétrica,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...


### Lista de Tickers

In [5]:
ticker_set = set()

for index, row in df_b3.iterrows():
    for ticker in row['tickers']:
        ticker_set.add(ticker)
        
ticker_list = list(ticker_set)
ticker_list.sort()

In [6]:
ticker_list[:5]

['AALR3', 'ABCB4', 'ABEV3', 'AERI3', 'AESB3']

### Lista de Nomes de Pregão

In [7]:
company_names_list = list()

for index, row in df_b3.iterrows():
    company_names_list.append(row['nome_pregao'])
        
company_names_list.sort()

In [9]:
company_names_list[:5]

['3R PETROLEUM', '3TENTOS', '524 PARTICIP', 'ABC BRASIL', 'ACO ALTONA']

---

### Dados do Portal de Notícias Suno Research

In [10]:
with open('../crawlers/suno/data/latest-results-suno.json', encoding='utf8') as json_file:
    data = json.load(json_file) 
    
df_suno = pd.DataFrame(data)

In [11]:
# Convert date to datetime format
df_suno['date'] = pd.to_datetime(df_suno['date'], format='%d/%m/%Y %H:%M')

# Set date column as index
df_suno.set_index('date', inplace=True)

# Order by date
df_suno.sort_index(inplace=True)

In [12]:
df_suno.sample(5)

,topic,title,full_text,url,tags
date,,,,,
2021-09-03 17:50:00,Mercado,Dólar fecha estável após payroll frustrar expe...,O dólar encerrou as negociações desta sexta-fe...,https://www.suno.com.br/noticias/dolar-fecha-e...,"[dólar, Especialistas]"
2021-09-06 17:31:00,Negócios,Ibovespa: Minerva (BEEF3) dispara e puxa alta ...,Mesmo com anúncio do Ministério da Agricultura...,https://www.suno.com.br/noticias/minerva-beef3...,"[Agências, BRF (BRFS3), Ibovespa, JBS (JBSS3),..."
2021-09-03 11:08:00,Mercado,Dólar recua após dados de emprego fracos nos EUA,O dólar opera em baixa na manhã desta sexta-fe...,https://www.suno.com.br/noticias/dolar-recua-a...,"[dólar, dólar futuro]"
2021-09-06 10:09:00,Internacional,China pode relaxar política monetária e gastar...,"Segundo dados recentes da Fitch Ratings, a Chi...",https://www.suno.com.br/noticias/china-politic...,"[Agências, Alemanha, China, Saudi Aramco]"
2021-09-03 20:07:00,Mercado Financeiro,"Após amargar perdas, Mauá Capital reduz área d...","Em informe nesta sexta-feira (3), a Mauá Capit...",https://www.suno.com.br/noticias/apos-amargar-...,"[Fundos multimercado, Mauá Capital]"


In [13]:
df_suno.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 61 entries, 2021-09-03 11:08:00 to 2021-09-07 11:42:00
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   topic      61 non-null     object
 1   title      61 non-null     object
 2   full_text  61 non-null     object
 3   url        61 non-null     object
 4   tags       61 non-null     object
dtypes: object(5)
memory usage: 2.9+ KB


---

### Método Baseado em Regras para NER

In [14]:
tz = pytz.timezone('America/Sao_Paulo')

for index, row in df_suno.iterrows():
    # Lista de palavras, sem pontuação, com letras maiúsculas
    list_of_words = [[w.upper().translate(str.maketrans('', '', string.punctuation))] for w in str(row['title']).split()]
    list_of_tags = [re.findall(r"[\w']+|[.,!?;]", x.upper()) for x in row['tags']]

    list_of_words.extend(list_of_tags)

    # Extrair o ticker a partir dos nomes do pregão 
    # que aparecem nas tags (Exemplo: 'Vale' --> 'VALE3')
    company_names_set = set()

    for text_chunk in row['tags']:
        if text_chunk.upper() in company_names_list:            
            # Find index in df_b3
            # Empty if not found
            print (text_chunk.upper())
            index_list = df_b3.index[df_b3['nome_pregao'] == str(text_chunk.upper())].tolist()

            if index_list:
                index_number = index_list[0]
                tickers = df_b3['tickers'].loc[index_number]

                for ticker in tickers:
                    # Add ticker to the set
                    company_names_set.add(ticker)

    # Remover pontuação
    flat_list = [item.translate(str.maketrans('', '', string.punctuation)) for sublist in list_of_words for item in sublist]

    mentioned_tickers = set(flat_list).intersection(ticker_list)
    mentioned_tickers.update(company_names_set)

    if mentioned_tickers:
        print (f'Headline: {str(row.title)}')
        for ticker in list(mentioned_tickers):
            print (f'(datetime, ticker, topic) values ({index.replace(tzinfo=tz)}, {ticker}, {row.topic})') 
        print ('-'*35)

Headline: Aliansce (ALSO3) investirá na startup Hubsell para impulsionar e-commerce
(datetime, ticker, topic) values (2021-09-03 17:46:00-03:00, ALSO3, Negócios)
-----------------------------------
Headline: Ibovespa sobe 0,2% após payroll; Magazine Luiza (MGLU3) dispara 5%
(datetime, ticker, topic) values (2021-09-03 17:55:00-03:00, EQTL3, Mercado)
(datetime, ticker, topic) values (2021-09-03 17:55:00-03:00, MGLU3, Mercado)
-----------------------------------
Headline: Nove empresas disputam rodada de licitações da ANP; veja lista
(datetime, ticker, topic) values (2021-09-03 19:00:00-03:00, PETR4, Economia)
-----------------------------------
Headline: Rede D’Or (RDOR3) pagará R$ 2,128 bi em dividendos e JCP
(datetime, ticker, topic) values (2021-09-03 19:11:00-03:00, RDOR3, Negócios)
-----------------------------------
Headline: Fundo imobiliário SNFF11 anuncia desconto na parcela da taxa de administração
(datetime, ticker, topic) values (2021-09-03 22:21:00-03:00, BPAC11, Negócios)


---